In [1]:
import cv2
import numpy as np
import os
from random import shuffle

import tensorflow as tf

C:\Users\Fernando\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [74]:
TRAIN_DIR = './train' 
TEST_DIR = './test'

IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = "dogsvscats-{}-{}.model".format(LR, '2conv-pool-3-channels')
TRAIN_DATA_FILE = 'sentdex.training.npy'
TEST_DATA_FILE = 'sentdex.test.npy'
CHANNELS = 3

In [38]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat' : return [1,0]
    else : return [0,1]


In [39]:
def create_train_data():
    training_data = []
    for img in os.listdir(TRAIN_DIR) :
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
        
    shuffle(training_data)
    np.save(TRAIN_DATA_FILE, training_data)
    return training_data

In [40]:
def process_test_data():
    testing_data = []
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    shuffle(testing_data)
    np.save(TEST_DATA_FILE, testing_data)
    return testing_data


In [41]:
train_data = create_train_data()

In [7]:
train_data = np.load(TRAIN_DATA_FILE)

In [75]:
#def model_fn(features, labels, mode):
def model_fn() :
    """Model function for CNN."""
    # Input Layer
    
    
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(32,5,activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS)))
    model.add(tf.keras.layers.MaxPool2D(2))
    
    model.add(tf.keras.layers.Conv2D(64,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(2))
    
    
    model.add(tf.keras.layers.Conv2D(128,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    
    model.add(tf.keras.layers.Conv2D(64,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    
    model.add(tf.keras.layers.Conv2D(32,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    
    
    return model

In [44]:
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,CHANNELS)
Y = np.array([i[1] for i in train])

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,CHANNELS)
test_y = np.array([i[1] for i in test])


In [76]:
model = model_fn()

model.compile(
    loss=tf.keras.losses.mean_squared_error,
    optimizer=tf.keras.optimizers.Adadelta(LR),
    metrics=['accuracy']
)

In [82]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs' + MODEL_NAME, histogram_freq=0,
                          write_graph=True, write_images=False)

model.fit(
    X, #inputs
    Y, #features
    epochs=100, 
    batch_size=25,
    validation_data=(test_x,test_y),
    shuffle=True,
    callbacks=[tensorboard]
)

Train on 24500 samples, validate on 500 samples
Epoch 1/100
24500/24500 [==============================] - 21s 868us/step - loss: 0.0949 - acc: 0.8785 - val_loss: 0.1775 - val_acc: 0.7680
Epoch 2/100
24500/24500 [==============================] - 21s 846us/step - loss: 0.0948 - acc: 0.8804 - val_loss: 0.1795 - val_acc: 0.7640
Epoch 3/100
24500/24500 [==============================] - 21s 844us/step - loss: 0.0938 - acc: 0.8816 - val_loss: 0.1790 - val_acc: 0.7620
Epoch 4/100
24500/24500 [==============================] - 21s 860us/step - loss: 0.0932 - acc: 0.8832 - val_loss: 0.1768 - val_acc: 0.7680
Epoch 5/100
24500/24500 [==============================] - 21s 858us/step - loss: 0.0933 - acc: 0.8814 - val_loss: 0.1790 - val_acc: 0.7620
Epoch 6/100
24500/24500 [==============================] - 21s 847us/step - loss: 0.0925 - acc: 0.8831 - val_loss: 0.1777 - val_acc: 0.7740
Epoch 7/100
24500/24500 [==============================] - 21s 846us/step - loss: 0.0922 - acc: 0.8824 - val_los

KeyboardInterrupt: 

In [83]:
model.save(MODEL_NAME)
#model = tf.keras.models.load_model(MODEL_NAME)

In [54]:
test_data_label = process_test_data()
test_data = np.array([i[0] for i in test_data_label]).reshape(-1,IMG_SIZE,IMG_SIZE,CHANNELS)
test_label = [i[1] for i in test_data_label]

In [80]:
prediction = model.predict(test_data)

In [81]:
template = '{},{}\n'
submission_file = 'submission_file_keras-bigger-3-channels.csv'
with open(submission_file,'w') as f:
    f.write('id,label\n')
            
with open(submission_file,'a') as f:
    for label, pred in zip(test_label, prediction) :
        f.write(template.format(label, pred[1]))

In [14]:
import matplotlib.pyplot as plt

#fig = plt.figure()
im = test_x[0]
plt.imshow(im.reshape(im.shape[0], im.shape[1]))

In [15]:
eval_x = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = np.array([i[1] for i in test])